## Problem 1

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.4.1


## Problem 2
### MNIST dataset

In [2]:
(X_train, y_train) , (X_test, y_test)= keras.datasets.fashion_mnist.load_data()

X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

In [3]:
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [4]:
model = keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(64)

In [7]:
history = model.fit(
    train_dataset,
    batch_size=64,
    epochs=10,
    validation_data=val_dataset
    )

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.6607 - sparse_categorical_accuracy: 0.7686 - val_loss: 0.4203 - val_sparse_categorical_accuracy: 0.8515
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.3971 - sparse_categorical_accuracy: 0.8570 - val_loss: 0.3801 - val_sparse_categorical_accuracy: 0.8644
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.3477 - sparse_categorical_accuracy: 0.8722 - val_loss: 0.3517 - val_sparse_categorical_accuracy: 0.8740
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.3173 - sparse_categorical_accuracy: 0.8822 - val_loss: 0.3351 - val_sparse_categorical_accuracy: 0.8798
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2954 - sparse_categorical_accuracy: 0.8900 - val_loss: 0.3328 - val_sparse_categorical_accuracy: 0.8759
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2806 - sparse_categorica

In [8]:
print('Evaluate on test data')
results = model.evaluate(X_test, y_test, batch_size=128)
predictions = model.predict(X_test)

print('\nPredicted label vs. actual label')
for i in range(6):
    predicted_label, actual_label = np.argmax(predictions[i]), int(y_test[i])
    print('{} vs. {}'.format(predicted_label, actual_label))

Evaluate on test data
79/79 [==============================] - 0s 1ms/step - loss: 0.3601 - sparse_categorical_accuracy: 0.8749

Predicted label vs. actual label
9 vs. 9
2 vs. 2
1 vs. 1
1 vs. 1
6 vs. 6
1 vs. 1


### 20NG Dataset

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train = fetch_20newsgroups(subset='train')
X_test = fetch_20newsgroups(subset='test')
y_train = X_train.target
y_test = X_test.target

LABELS = set(y_train)

X_train = vectorizer.fit_transform(X_train.data).todense()
X_test = vectorizer.transform(X_test.data).todense()
print('X_train shape -', X_train.shape)
print('y_train shape -', y_train.shape)
print('X_test shape -', X_test.shape)
print('y_test shape -', y_test.shape)
print('Labels -', LABELS)

X_train shape - (11314, 130107)
y_train shape - (11314,)
X_test shape - (7532, 130107)
y_test shape - (7532,)
Labels - {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [3]:
model = keras.Sequential([
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(20, activation="softmax")
])

In [4]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [5]:
history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=6,
    )

Epoch 1/6
177/177 [==============================] - 17s 91ms/step - loss: 2.6811 - sparse_categorical_accuracy: 0.4063
Epoch 2/6
177/177 [==============================] - 14s 80ms/step - loss: 0.8608 - sparse_categorical_accuracy: 0.9163
Epoch 3/6
177/177 [==============================] - 14s 80ms/step - loss: 0.2438 - sparse_categorical_accuracy: 0.9678
Epoch 4/6
177/177 [==============================] - 16s 91ms/step - loss: 0.0898 - sparse_categorical_accuracy: 0.9866
Epoch 5/6
177/177 [==============================] - 19s 106ms/step - loss: 0.0418 - sparse_categorical_accuracy: 0.9935
Epoch 6/6
177/177 [==============================] - 19s 108ms/step - loss: 0.0183 - sparse_categorical_accuracy: 0.9964


In [6]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
predictions = model.predict(X_test)

print('\nPredicted label vs. actual label')
for i in range(9):
    predicted_label, actual_label = np.argmax(predictions[i]), int(y_test[i])
    print('{} vs. {}'.format(predicted_label, actual_label))

Evaluate on test data
59/59 [==============================] - 6s 75ms/step - loss: 0.5691 - sparse_categorical_accuracy: 0.8517

Predicted label vs. actual label
7 vs. 7
5 vs. 5
0 vs. 0
17 vs. 17
0 vs. 19
13 vs. 13
15 vs. 15
2 vs. 15
5 vs. 5


In [9]:
predicted_labels = []

for i in range(len(predictions)):
    predicted_labels.append(np.argmax(predictions[i]))

In [11]:
from sklearn.metrics import accuracy_score

accuracy_score(predicted_labels, y_test)

0.8516994158258099